In [39]:
import pandas as pd
import plotly.express as px
import numpy as np

In [32]:
df = pd.read_csv("matmul_results_2.csv",sep =";")
df_jit = df[df["file"] == "JIT_1"].groupby(['N','M','K']).min().reset_index()
df_nojit = df[df["file"] == "NOJIT_1"].groupby(['N','M','K']).min().reset_index()
df_jit

,N,M,K,file,TOT_CYC,L1_DCM,TOT_INS,Unnamed: 7
0,1,1,1,JIT_1,791,19,354,NaN
1,1,1,2,JIT_1,800,17,358,NaN
2,1,1,4,JIT_1,915,20,366,NaN
3,1,1,8,JIT_1,843,15,382,NaN
4,1,1,16,JIT_1,1075,23,415,NaN
...,...,...,...,...,...,...,...,...
507,128,128,8,JIT_1,403956,2522,864478,NaN
508,128,128,16,JIT_1,820619,3142,1730782,NaN
509,128,128,32,JIT_1,1628841,59061,3467101,NaN
510,128,128,64,JIT_1,3243912,141030,6932317,NaN


In [50]:
compare_df = pd.merge(df_jit,df_nojit, on = ['N','M','K'],suffixes = ("_JIT","_NOJIT"))
compare_df = compare_df.drop(["file_JIT","file_NOJIT","Unnamed: 7_JIT","Unnamed: 7_NOJIT"],axis = 1)


In [54]:
def compare_results(df,col_name):
    compare_df = df.copy()
    compare_df["compare_"+col_name] = (compare_df[col_name+"_NOJIT"] - compare_df[col_name +"_JIT"]) / compare_df[col_name +"_NOJIT"] * 100
    compare_df[col_name +"_adv_JIT"] = np.where(compare_df["compare_"+col_name] > 0,1, -1)
    compare_df["compare_"+col_name] = np.abs(compare_df["compare_"+col_name])
    return compare_df
adv_df = compare_results(compare_df,"TOT_CYC")
adv_df = compare_results(adv_df, "L1_DCM")
adv_df = compare_results(adv_df,"TOT_INS")

,N,M,K,TOT_CYC_JIT,L1_DCM_JIT,TOT_INS_JIT,TOT_CYC_NOJIT,L1_DCM_NOJIT,TOT_INS_NOJIT,compare_TOT_CYC,TOT_CYC_adv_JIT,compare_L1_DCM,L1_DCM_adv_JIT,compare_TOT_INS,TOT_INS_adv_JIT
0,1,1,1,791,19,354,918,21,382,13.834423,1,9.523810,1,7.329843,1
1,1,1,2,800,17,358,909,24,396,11.991199,1,29.166667,1,9.595960,1
2,1,1,4,915,20,366,913,23,424,0.219058,-1,13.043478,1,13.679245,1
3,1,1,8,843,15,382,1062,17,480,20.621469,1,11.764706,1,20.416667,1
4,1,1,16,1075,23,415,1221,34,592,11.957412,1,32.352941,1,29.898649,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,128,128,8,403956,2522,864478,402163,2525,863592,0.445839,-1,0.118812,1,0.102595,-1
508,128,128,16,820619,3142,1730782,816530,3237,1725801,0.500778,-1,2.934816,1,0.288620,-1
509,128,128,32,1628841,59061,3467101,1625444,58967,3450216,0.208989,-1,0.159411,-1,0.489390,-1
510,128,128,64,3243912,141030,6932317,3239243,140995,6899048,0.144139,-1,0.024824,-1,0.482226,-1


In [59]:
adv_df[['TOT_CYC_adv_JIT','L1_DCM_adv_JIT','TOT_INS_adv_JIT']].value_counts()

TOT_CYC_adv_JIT  L1_DCM_adv_JIT  TOT_INS_adv_JIT
 1                1               1                 225
                 -1               1                 127
-1                1               1                  49
                                 -1                  38
                 -1               1                  32
                                 -1                  24
 1                1              -1                  10
                 -1              -1                   7
dtype: int64

In [64]:
def prompt_results(df,col,fixed_param):
    nmk = ['N','M','K']
    nmk.remove(fixed_param)
    for k in df[fixed_param].unique():
        fig = px.scatter(df[df[fixed_param] == k], x = nmk[0], y = nmk[1], size = "compare_" + col, color = col +"_adv_JIT", title = "Comparaison " + col +" JIT/NOJIT for " +fixed_param+"  = "+str(k))
        fig.show()
prompt_results(adv_df,"TOT_CYC","K")
prompt_results(adv_df,"L1_DCM","K")
prompt_results(adv_df,"TOT_INS","K")
prompt_results(adv_df,"TOT_CYC","M")